In [1]:
# Instalar detectron2
#!pip install segmentation-models-pytorch

In [2]:
# Importar librerías
import torch
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from PIL import Image
import numpy as np


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class UrbanDataset(Dataset):
    def __init__(self, img_paths, mask_paths, transform=None):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        image = np.array(Image.open(self.img_paths[idx]).convert("RGB"))
        mask = np.array(Image.open(self.mask_paths[idx]).convert("L"))
        mask = mask / 255.0
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented['image'], augmented['mask']
        return torch.tensor(image).permute(2,0,1).float(), torch.tensor(mask).unsqueeze(0).float()


In [7]:
import glob

train_img_paths = sorted(glob.glob("dataset/train/images/*.png"))
train_mask_paths = sorted(glob.glob("dataset/train/masks/*.png"))

val_img_paths = sorted(glob.glob("dataset/val/images/*.png"))
val_mask_paths = sorted(glob.glob("dataset/val/masks/*.png"))


In [9]:
train_dataset = UrbanDataset(train_img_paths, train_mask_paths)
val_dataset = UrbanDataset(val_img_paths, val_mask_paths)


In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [11]:
for epoch in range(10):
    for images, masks in train_loader:
        preds = model(images)
        loss = loss_fn(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


OSError: image file is truncated

In [8]:
# Definir modelo
model = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=1)
loss_fn = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Entrenamiento simplificado
for epoch in range(10):
    for images, masks in DataLoader(train_dataset, batch_size=4):
        preds = model(images)
        loss = loss_fn(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

NameError: name 'train_dataset' is not defined